In [ ]:
!pip install transformers==4.30.2 peft==0.4.0
!pip install sentencepiece
!pip install accelerate
!pip install torch
!pip install peft
!pip install datasets
!pip install bitsandbytes

In [2]:
!git clone https://github.com/AI4Finance-Foundation/FinNLP.git

import sys
sys.path.append('/content/FinNLP/')

Cloning into 'FinNLP'...
remote: Enumerating objects: 1310, done.
remote: Counting objects: 100% (369/369), done.
remote: Compressing objects: 100% (166/166), done.
remote: Total 1310 (delta 214), reused 296 (delta 169), pack-reused 941
Receiving objects: 100% (1310/1310), 4.21 MiB | 8.19 MiB/s, done.
Resolving deltas: 100% (587/587), done.


In [3]:
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM

from peft import PeftModel
import torch

from finnlp.benchmarks.fpb import test_fpb
from finnlp.benchmarks.fiqa import test_fiqa , add_instructions
from finnlp.benchmarks.tfns import test_tfns
from finnlp.benchmarks.nwgi import test_nwgi


In [ ]:
# Load Model
base_model = "THUDM/chatglm2-6b"
peft_model = "oliverwang15/FinGPT_v31_ChatGLM2_Sentiment_Instruction_LoRA_FT"
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
model = AutoModel.from_pretrained(base_model, trust_remote_code=True, load_in_8bit = True, device_map = "auto")
model = PeftModel.from_pretrained(model, peft_model)
model = model.eval()

In [5]:
batch_size = 8

In [6]:
# FPB
res = test_fpb(model, tokenizer, batch_size = batch_size)

Generating train split:   0%|          | 0/4846 [00:00<?, ? examples/s]



Prompt example:
Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.
Input: L&T has also made a commitment to redeem the remaining shares by the end of 2011 .
Answer: 


Total len: 1212. Batchsize: 8. Total steps: 152


100%|██████████| 152/152 [01:30<00:00,  1.67it/s]

Acc: 0.8531353135313532. F1 macro: 0.8355311905054061. F1 micro: 0.8531353135313532. F1 weighted (BloombergGPT): 0.8518766219402919. 


In [7]:
# FiQA
res = test_fiqa(model, tokenizer, prompt_fun = add_instructions, batch_size = batch_size)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]



Prompt example:
Instruction: What is the sentiment of this tweet? Please choose an answer from {negative/neutral/positive}.
Input: This $BBBY stock options trade would have more than doubled your money https://t.co/Oa0loiRIJL via @TheStreet
Answer: 


Total len: 275. Batchsize: 8. Total steps: 35


100%|██████████| 35/35 [00:18<00:00,  1.93it/s]

Acc: 0.8327272727272728. F1 macro: 0.7457078313253013. F1 micro: 0.8327272727272728. F1 weighted (BloombergGPT): 0.8475613362541076. 


In [8]:
# TFNS
res = test_tfns(model, tokenizer, batch_size = batch_size)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]



Prompt example:
Instruction: What is the sentiment of this tweet? Please choose an answer from {negative/neutral/positive}.
Input: $ALLY - Ally Financial pulls outlook https://t.co/G9Zdi1boy5
Answer: 


Total len: 2388. Batchsize: 8. Total steps: 299


100%|██████████| 299/299 [02:33<00:00,  1.95it/s]

Acc: 0.876465661641541. F1 macro: 0.8422150651552788. F1 micro: 0.876465661641541. F1 weighted (BloombergGPT): 0.8760116165894634. 


In [9]:
# NWGI
res = test_nwgi(model, tokenizer, batch_size = batch_size)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/16184 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4047 [00:00<?, ? examples/s]



Prompt example:
Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.
Input: In the latest trading session, Adobe Systems (ADBE) closed at $535.98, marking a +0.31% move from the previous day.
Answer: 


Total len: 4047. Batchsize: 8. Total steps: 506


100%|██████████| 506/506 [05:31<00:00,  1.53it/s]

Acc: 0.6375092661230541. F1 macro: 0.6449413175330677. F1 micro: 0.6375092661230541. F1 weighted (BloombergGPT): 0.6368355381269861. 
